In [1]:
# Imports
using Distributions
using LinearAlgebra
using DataFrames, CSV
using Distributions
using Random

In [3]:
global T_timescale = 78 ## 1.5 years
global T_monitoring = 24 ## duration of monitoring period in weeks
global N_cohort = 3
global Quota = N_cohort*500

mutable struct Date
    id::Integer ## day of the year
    X::AbstractFloat ## This could be a date fixed effect or a vector of date characteristics
end

# mutable struct Cohort
#     id::Integer ## Cohort ID
#     min_date::Integer ## First date anyone in the cohort can enter; Assume time is indexed from 0 to 365
#     max_date::Integer ## Laste date anyone in the cohort can enter; Note -- cohort entries may overlap (maybe...)
# end

# mutable struct Trip
#     date::Integer ## date of trip
#     departure::AbstractFloat ## time of departure
#     duration::AbstractFloat ## trip duration
#     distance::AbstractFloat ## trip distance
# end

# mutable struct Rider
#     id::Integer ## rider ID
#     cohort_id::Integer ## rider's cohord ID
#     X::Vector{AbstractFloat}  ## rider characteristics
#     init_date::Integer ## Rider's date of entry
#     trips::Vector{Trip}
# end

mutable struct BasicOutcome
    date::Integer ## date of trip
    y::AbstractFloat ## outcome
end

mutable struct BasicRider
    id::Integer ## rider ID
    cohort_id::Integer ## rider's cohord ID -- in order of the cohort array
    X::Vector{AbstractFloat}  ## rider characteristics
    init_date::Integer ## Rider's date of entry
    min_date::Integer ## First date anyone in the cohort can enter; Assume time is indexed from 0 to 365
    max_date::Integer ## Laste date anyone in the cohort can enter; Note -- cohort entries may overlap (maybe...)
    outcomes::Vector{BasicOutcome} ## outcome draws
end

mutable struct BasicCohort
#     id::Integer ## Cohort ID
    init_coeff::AbstractFloat ## alpha_t for the cohort
end


In [4]:
### Defining Statistical Distributions ##

## Set seed
rng_seed = MersenneTwister(1234);

## Multidimensional Gaussian
function get_mvnormal_dist(d = 5, η = 1)
    # Dimensionality
    # d = 5
    # η = 1 # Uniform distribution over correlation matrices

    # Distribution setup
    Ω_dist = LKJ(d, η)
    σ_dist = InverseGamma(2, 3)
    μ_dist = MvNormal(5, 1)

    # Draw values
    Ω = rand(Ω_dist)                 # Correlation matrix
    σ = sqrt.(rand(σ_dist, d))       # Individual standard deviations
    μ = rand(μ_dist)                 # Mean vector

    # Create covariance matrix
    # Need to specify Symmetric(...) to enforce symmetry, otherwise
    # possible non-Hermitian error.
    Σ = Symmetric(diagm(σ)*Ω*diagm(σ))

    # Create random mean/variance distribution
    mvnormal_dist = MvNormal(μ, Σ)
    return(mvnormal_dist)
end

## Standard Gumbel errors
standard_gumbel_dist = Gumbel()

Gumbel{Float64}(μ=0.0, θ=1.0)

In [6]:
## Recruitment Distribution ##

### Idea: Each recruit must be added between time 0 and the end of the year: p \in (0, 1) * 365
###       Draw t = inv_logit(alpha_t + beta_t*X)
###       Outcomes are O_it ~ f(mu*X_i, gamma*X_t)


function inv_logit(x)
   return exp.(x)./(1 .+ exp.(x)); 
end

function logit(p)
   return log.(p ./ (1 .- p)) 
end

logit_scaler = 25 ## logit is most potent on the support -5 to 5, so we can either choose imputs more intelligently or choose a scaler

function draw_init_date!(rider, β_t)
    t_i = round(inv_logit((cohort_array[rider.cohort_id].init_coeff + β_t'*rider.X)/logit_scaler) * T_timescale);
    rider.init_date = t_i;
    rider.min_date = max(t_i - T_monitoring,0);
    rider.max_date = min(t_i + T_monitoring, T_timescale);
end

# function linear_outcomes()

draw_init_date! (generic function with 1 method)

In [7]:
## Instantiate cohort
cohort_array = Array{BasicCohort}(undef, N_cohort);
cohort_init_dist = [15, 40, 60] # ## we can simulate this instead: cohort_init_dist = Uniform(0,T_timescale) 

for c in 1:N_cohort
    draw = cohort_init_dist[c]
    cohort_array[c] = BasicCohort(draw)
end

## Instantiate Dates
date_array = Array{Date}(undef, T_timescale);
date_fe_dist = Normal(0, 2);

for t in 1:T_timescale
   date_array[t] = Date(t, rand(rng_seed, date_fe_dist))
end


## Instantiate Individuals
panel_array = Array{BasicRider}(undef, Quota);
indiv_X_dist = get_mvnormal_dist()
init_date_dist = Normal(0,2);
β_t_draw = rand(init_date_dist, 5);

N_per_cohort = Integer(floor(Quota/N_cohort));
for i in 1:N_per_cohort
    for c in 1:N_cohort
        new_rider = BasicRider(i, c, rand(rng_seed, indiv_X_dist), 0, 0, 0, [])
        draw_init_date!(new_rider, β_t_draw);
        
        panel_array[N_per_cohort*(c-1)+i] = new_rider
    end
end



In [24]:
indiv_outcome_dist = Normal(0.2,0.5);
date_outcome_dist = Normal(0,0.1);
treatment_dist = Normal(-1, 0.1);

β_i_draw = rand(rng_seed, indiv_outcome_dist, 5);
γ_t_draw = rand(rng_seed, date_outcome_dist);
τ_draw = rand(rng_seed, treatment_dist);


In [23]:
for i in 1:Quota
    indiv_effect = β_i_draw'*rider.X;
    treatment_effect = τ_draw * inv_logit(β_i_draw'*rider.X);
    
    outcomes_before_treatment = [BasicOutcome(t,indiv_effect + γ_t_draw'*date_array[t].X) for t=rider.min_date:rider.init_date];
    outcomes_after_treatment = [BasicOutcome(t,indiv_effect + γ_t_draw'*date_array[t].X + treatment_effect) for t=(rider.init_date+1):rider.max_date];
    
    panel_array[i].outcomes = vcat(outcomes_before_treatment, outcomes_after_treatment);
    
end